# Bullet Forge Fine-tuning

This notebook is used for fine-tuning the Bullet Forge model in Open AI's API. Please refer to the README for more details.

## Step 1: Provide Required Arguments
Make sure to provide the following arguments:
- JSONL_FILE: The path to the JSONL file containing the training data.
- BASE_MODEL: The base model to use for fine-tuning.

In [4]:
JSONL_FILE = input("Enter the path to the JSON file: ")
print(f"==> Provided JSONL file path: {JSONL_FILE}")
BASE_MODEL = input("Enter the base model: ")
print(f"==> OpenAI base model: {BASE_MODEL}")
FINE_TUNED_MODEL_NAME = input("Enter fine-tuned model suffix: ")
print(f"==> Fine-tuned model suffix: {FINE_TUNED_MODEL_NAME}")

==> Provided JSONL file path: ./example-data.jsonl
==> OpenAI base model: ada-003-text
==> Fine-tuned model suffix: example-data


## Step 2: Check Training Data

Prepare the training data by checking on its validity and effectiveness using the provided JSONL file.

In [5]:
import subprocess

print("==> Preparing training data...")
if not JSONL_FILE:
    print("==> Error: Provided JSONL file or path invalid")
    exit
PREPARED_DATA = subprocess.run(['openai', 'tools', 'fine_tunes.prepare_data', '-f', JSONL_FILE], capture_output=True, text=True)
print(PREPARED_DATA.stdout)
if PREPARED_DATA.returncode == 0:
    print("==> Training data looks good!")
else:
    print("==> Error: Failed to analyze training data.")
    print(f"==> Error: {PREPARED_DATA.stderr}")


==> Preparing training data...
Analyzing...

- Your file contains 3 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples
- All prompts end with suffix `ation>`
- All prompts start with prefix `<prompt for `
==> Error: Failed to analyze training data.
==> Error: 

ERROR in common_suffix validator: All completions are identical: `<ideal generated bullet>`
Ensure completions are different, otherwise the model will just repeat `<ideal generated bullet>`

Aborting...


## Step 3: Provide OpenAI API Key

Make sure to provide the following arguments:
- OPENAI_API_KEY: Your OpenAI API key

**_WARNING_**: Do not expose your OpenAI API key to the public repository.

In [6]:
import openai

OPENAI_API_KEY = input("Enter your OpenAI API Key: ")
MASKED_OPENAI_API_KEY = "*" * len(OPENAI_API_KEY) if OPENAI_API_KEY else "Error: No key found!"

openai.api_key = OPENAI_API_KEY
print("==> API_KEY:", MASKED_OPENAI_API_KEY)

==> API_KEY: ***************************************************


## Step 4: Create Fine-Tuning File

Upload files to your instance of OpenAI API prior to running the fine-tuning job. This helps catch mistakes within the training file prior to running a fine-tuning job and persists the data in your API instance.

In [7]:
import openai

print("==> Starting training data upload job...")
if not PREPARED_DATA.returncode == 0:
    exit
print("==> Training data upload job started...")

response = openai.File.create(
    file=open(JSONL_FILE, encoding="utf-8"),
    purpose="fine-tune"
)

if response.status == "uploaded":
    print("==> Training data uploaded")
else:
    print("==> Failed to upload training data.")
print(f"==> Response object: ${response}")

FILE_ID = response["id"]
print(f"==> File ID: ${FILE_ID}")

==> Starting training data upload job...
==> Training data upload job started...
==> Training data uploaded
==> Response object: ${
  "object": "file",
  "id": "file-hvTwdbDdNcpeAxGDBAABRnIM",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 282,
  "created_at": 1688161142,
  "status": "uploaded",
  "status_details": null
}
==> File ID: $file-hvTwdbDdNcpeAxGDBAABRnIM


_**Deleting Uploaded Fine-Tuning Training Files**_: There may be times when you want to delete training files from your API instance.

In [16]:
import openai

response = openai.File.delete(sid=FILE_ID)

if response.deleted:
    print("==> Training data deleted!")
else:
    print("==> Error: Failed to delete training data.")

print(f"==> Response object: ${response}")

==> Training data deleted!
==> Response object: ${
  "object": "file",
  "id": "file-g0a8t3jycHrBF62RKhrq7NAD",
  "deleted": true
}


## Step 5: Start the Fine-Tuning Job

Once the request is made to the API, the fine-tuning may take a while to complete. The following will check for completion periodically.

In [8]:
import openai

print("==> Starting fine-tuning job...")
if not FILE_ID:
    print("==> Error: Provided FILE_ID invalid")
    exit
if not BASE_MODEL:
    print("==> Error: Provided BASE_MODEL invalid")
    exit
print("==> Fine-tuning job started.")

response = openai.FineTune.create(
  model=BASE_MODEL,
  training_file=FILE_ID,
  suffix = FINE_TUNED_MODEL_NAME
)

print("==> Fine-tuning in-progress...")
if response.status_code == 200:
  print(f"Fine-tuned Model ID: {response.json()['id']}")
else:
  print(f"Error Occurred: {response.json()['error']}")
  exit

print("==> Fine-tuning completed!")

==> Starting fine-tuning job...
==> Fine-tuning job started.


InvalidRequestError: Billing hard limit has been reached

## Step 6: Use The Fine-Tuned Model

Fine-tuned model CLI usage: 

```bash
openai api completions.create -m <FINE_TUNED_MODEL> -p <YOUR PROMPT>
```

For usage of the fine-tuned model, please refer to the code within the `server/bulletForge/` directory.
